# Solución del Reto
**TC2008B.4 | Equipo 4**

**Integrantes:**
* Daniela Garza - A00829404
* Jorge Borbolla - A01383867
* Omar Pérez - A01383853
* Javier Agostini - A00827216

**Instrucciones**

A continuación, se muestra el código, así como la simulación que conforma la solución del reto del bloque TC2008B. Consiste en un sistema multiagentes que modela una intersección inteligente controlada por semáforos. 

El objetivo es plantear una solución a los problemas de vialidad que existen en México y el mundo, aprovechando la tecnología para reducir accidentes en las intersecciones.

In [1]:
!pip3 install mesa

     |████████████████████████████████| 668 kB 21.3 MB/s 
     |████████████████████████████████| 60 kB 6.8 MB/s 


## Imports

In [2]:
from mesa import Agent, Model 
from mesa.space import MultiGrid
from mesa.time import SimultaneousActivation
from mesa.datacollection import DataCollector

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128
from matplotlib.colors import LinearSegmentedColormap

import numpy as np
import pandas as pd
import random

import time
import datetime

## Crear el modelo

In [3]:
randomDireccion=0
switchSemaforo=False
auxSwitch=False
movimientosT=0

#Función que te devuelve la habitación (calle e intersección), ayuda a graficar el resultado
def obtener_habitacion(modelo):
    habitacion = np.zeros((modelo.grid.width, modelo.grid.height))
    for celda in modelo.grid.coord_iter():
        contenido_celda, x, y = celda
        for contenido in contenido_celda:
            if isinstance(contenido, Coche):
                habitacion[x][y] = 5
            else:
                if isinstance(contenido,Semaforo):
                    habitacion[x][y]=contenido.estadoS
                else:
                    habitacion[x][y] = contenido.estado
    return habitacion
    

#Agente Coche---------------------------------------------------------------------------------------------------------------------------------------------------------------
class Coche(Agent):
    #Constructor del agente Coche, el cual inicializa sus valores
    def __init__(self, id_unico, modelo):
        super().__init__(id_unico, modelo)
        self.id_unico=id_unico
        self.nueva_posicion = None
        self.movimientos = 0
        self.sentidoCOCHE=None
  
    #Función que define el estado que tendrá el coche para la siguiente iteración sin asignarlo rodavía
    #Esta función puede funcionar como un "pre-cambio" donde solo se asignan valores
    def step(self):
        global randomDireccion
        global auxSwitch

        vecinos = self.model.grid.get_neighbors(
            self.pos,
            moore=False,
            include_center=True)
        for vecino in vecinos:
            if isinstance(vecino,Coche) and self.sentidoCOCHE==vecino.sentidoCOCHE:
                self.nueva_posicion=self.pos
        for vecino in vecinos:
            #En caso de que haya un semáforo entre los vecinos
            if isinstance(vecino,Semaforo):
                #Si el semáforo esta en rojo o amarillo el auto se detiene
                if (vecino.estadoS==vecino.ROJO or vecino.estadoS==vecino.AMARILLO) and vecino.sentidoSEMAFORO==self.sentidoCOCHE:
                    self.nueva_posicion=self.pos
                    break
                #Si el semáforo se encuentra en verde el auto puede avanzar, también
                #escoge la dirección o camino que tomará
                if vecino.estadoS==vecino.VERDE and vecino.sentidoSEMAFORO==self.sentidoCOCHE:
                    randomDireccion=random.randint(1,2)
                    vecinos2 = self.model.grid.get_neighbors(
                        self.pos,
                        moore=False,
                        include_center=True)
                    for vecino2 in vecinos2:
                        if isinstance(vecino, Piso) and vecino.pos==self.pos:
                            vecino.siguiente_estado = vecino.estado
                            if vecino.siguiente_estado==vecino.CALLE:
                                #Se moverá del semáforo dependiendo que sentido tiene la calle
                                if vecino.SENTIDO_CALLE==vecino.SENTIDO_Este:
                                    self.nueva_posicion= tuple(sum(x) for x in zip(self.pos,(0,1)))
                                    self.sentidoCOCHE="E"
                                    break

                                if vecino.SENTIDO_CALLE==vecino.SENTIDO_Oeste:
                                    self.nueva_posicion= tuple(sum(x) for x in zip(self.pos,(0,-1)))
                                    self.sentidoCOCHE="O"
                                    break

                                if vecino.SENTIDO_CALLE==vecino.SENTIDO_Norte:
                                    self.nueva_posicion= tuple(sum(x) for x in zip(self.pos,(1,0)))
                                    self.sentidoCOCHE="N"
                                    break

                                if vecino.SENTIDO_CALLE==vecino.SENTIDO_Sur:
                                    self.nueva_posicion= tuple(sum(x) for x in zip(self.pos,(-1,0)))
                                    self.sentidoCOCHE="S"
                                    break
            else:
                #Si no exsiten semáforos cerca este se tendrá en cuenta como calle, donde el auto
                #puede circular sin restricciones
                if isinstance(vecino, Piso) and vecino.pos==self.pos:
                    vecino.siguiente_estado = vecino.estado
                    if vecino.siguiente_estado==vecino.CALLE:
                        #Se moverá en base al sentido de la calle
                        if vecino.SENTIDO_CALLE==vecino.SENTIDO_Este:
                            self.sentidoCOCHE="E"
                            self.nueva_posicion= tuple(sum(x) for x in zip(self.pos,(0,1)))
                                
                        if vecino.SENTIDO_CALLE==vecino.SENTIDO_Oeste:
                            self.sentidoCOCHE="O"
                            self.nueva_posicion= tuple(sum(x) for x in zip(self.pos,(0,-1)))
                            
                        if vecino.SENTIDO_CALLE==vecino.SENTIDO_Norte:
                            self.sentidoCOCHE="N"
                            self.nueva_posicion= tuple(sum(x) for x in zip(self.pos,(1,0)))
                             
                        if vecino.SENTIDO_CALLE==vecino.SENTIDO_Sur:
                            self.sentidoCOCHE="S"
                            self.nueva_posicion= tuple(sum(x) for x in zip(self.pos,(-1,0)))

                        if vecino.SENTIDO_CALLE==vecino.SENTIDO_TODOS:
                            #Vuelta continua a la derecha
                            if randomDireccion==1:
                                if self.sentidoCOCHE == "N":
                                    self.nueva_posicion=tuple(sum(x) for x in zip(self.pos,(0,-1)))
                                    self.sentidoCOCHE="O"
                                    vecinos3 = self.model.grid.get_neighbors(
                                        self.pos,moore=False,
                                        include_center=True)
                                    for vecino3 in vecinos3:
                                        if isinstance(vecino3,Piso) and vecino3.SENTIDO_CALLE==vecino3.SENTIDO_Oeste:
                                            auxSwitch=True
                                    break
                                if self.sentidoCOCHE == "O":
                                    self.nueva_posicion=tuple(sum(x) for x in zip(self.pos,(-1,0)))
                                    self.sentidoCOCHE="S"
                                    vecinos3 = self.model.grid.get_neighbors(
                                        self.pos,moore=False,
                                        include_center=True)
                                    for vecino3 in vecinos3:
                                        if isinstance(vecino3,Piso) and vecino3.SENTIDO_CALLE==vecino3.SENTIDO_Sur:
                                            auxSwitch=True
                                    break
                                if self.sentidoCOCHE == "S":
                                    self.nueva_posicion=tuple(sum(x) for x in zip(self.pos,(0,1)))
                                    self.sentidoCOCHE="E"
                                    vecinos3 = self.model.grid.get_neighbors(
                                        self.pos,moore=False,
                                        include_center=True)
                                    for vecino3 in vecinos3:
                                        if isinstance(vecino3,Piso) and vecino3.SENTIDO_CALLE==vecino3.SENTIDO_Este:
                                            auxSwitch=True
                                    break
                                if self.sentidoCOCHE == "E":
                                    self.nueva_posicion=tuple(sum(x) for x in zip(self.pos,(1,0)))
                                    self.sentidoCOCHE="N"
                                    vecinos3 = self.model.grid.get_neighbors(
                                        self.pos,moore=False,
                                        include_center=True)
                                    for vecino3 in vecinos3:
                                        if isinstance(vecino3,Piso) and vecino3.SENTIDO_CALLE==vecino3.SENTIDO_Norte:
                                            auxSwitch=True
                                    break
                            #El auto irá en la misma dirección a la que se encontraba
                            if randomDireccion==2:
                                if self.sentidoCOCHE == "E":
                                    self.sentidoCOCHE="E"
                                    self.nueva_posicion= tuple(sum(x) for x in zip(self.pos,(0,1)))
                                    vecinos3 = self.model.grid.get_neighbors(
                                        self.pos,moore=False,
                                        include_center=True)
                                    for vecino3 in vecinos3:
                                        if isinstance(vecino3,Piso) and vecino3.SENTIDO_CALLE==vecino3.SENTIDO_Este:
                                            auxSwitch=True

                                if self.sentidoCOCHE == "O":
                                    self.sentidoCOCHE="O"
                                    self.nueva_posicion= tuple(sum(x) for x in zip(self.pos,(0,-1)))
                                    vecinos3 = self.model.grid.get_neighbors(
                                        self.pos,moore=False,
                                        include_center=True)
                                    for vecino3 in vecinos3:
                                        if isinstance(vecino3,Piso) and vecino3.SENTIDO_CALLE==vecino3.SENTIDO_Oeste:
                                            auxSwitch=True

                                if self.sentidoCOCHE == "N":
                                    self.sentidoCOCHE="N"
                                    self.nueva_posicion= tuple(sum(x) for x in zip(self.pos,(1,0)))
                                    vecinos3 = self.model.grid.get_neighbors(
                                        self.pos,moore=False,
                                        include_center=True)
                                    for vecino3 in vecinos3:
                                        if isinstance(vecino3,Piso) and vecino3.SENTIDO_CALLE==vecino3.SENTIDO_Norte:
                                            auxSwitch=True

                                if self.sentidoCOCHE == "S":
                                    self.sentidoCOCHE="S"
                                    self.nueva_posicion= tuple(sum(x) for x in zip(self.pos,(-1,0)))
                                    vecinos3 = self.model.grid.get_neighbors(
                                        self.pos,moore=False,
                                        include_center=True)
                                    for vecino3 in vecinos3:
                                        if isinstance(vecino3,Piso) and vecino3.SENTIDO_CALLE==vecino3.SENTIDO_Sur:
                                            auxSwitch=True
                          
        
    def advance(self):
        global auxSwitch
        global switchSemaforo
        global movimientosT
        
        #Se actualiza la pocisión y la cantidad de movimientos
        if self.pos != self.nueva_posicion:
            self.movimientos = self.movimientos + 1
            movimientosT=movimientosT+1
        
        #Cambia el estado del semáforo en caso de haber un auto ahí
        if auxSwitch:
            switchSemaforo=False
        
          
        # Movemos el auto a su nueva posicion
        self.model.grid.move_agent(self, self.nueva_posicion)
    

#Agente Piso -----------------------------------------------------------------------------------------------------------------------------------------------
class Piso(Agent):
    
    CALLE = 1
    NO_CALLE = 0

    SENTIDO_Sur="S"
    SENTIDO_Norte="N"
    SENTIDO_Este="E"
    SENTIDO_Oeste="O"
    SENTIDO_TODOS="T"
    
    #Constructor del agente Piso, el cual inicializa sus valores
    def __init__(self, pos, modelo, estado=NO_CALLE):
        super().__init__(pos, modelo)
        self.x, self.y = pos
        self.estado = estado
        self.siguiente_estado = None
        self.SENTIDO_CALLE=None

#AGREGAR AGENTE SEMAFORO (TENDRA 3 estados (luces) ROJO=2 AMARILLO=3 VERDE=4)-----------------------------------------------------------------------------------------
class Semaforo(Agent):
    ROJO=2
    AMARILLO=3
    VERDE=4

    #Constructor del agente Semaforo, el cual inicializa sus valores
    def __init__(self,pos,modelo,estadoS=ROJO):
        super().__init__(pos,modelo)
        self.x,self.y=pos
        self.estadoS=estadoS
        self.siguienteEstadoS=None
        self.sentidoSEMAFORO=None

    #Función que definirá el color que tomará el semáforo, solo se pre-asignarán valores
    def step(self):
        global switchSemaforo

        vecinos = self.model.grid.get_neighbors(
              self.pos,
              moore=True,
              include_center=False)
        for vecinoSem in vecinos:
            if isinstance(vecinoSem,Coche) and self.sentidoSEMAFORO==vecinoSem.sentidoCOCHE and not switchSemaforo:
                auxSwitch=False
                switchSemaforo=True
                self.siguienteEstadoS=self.VERDE
                break
            else:
                vecinosR = self.model.grid.get_neighbors(
                  self.pos,
                  moore=True,
                  include_center=False)
                for vecinoR in vecinosR:
                    if not isinstance(vecinoR,Coche) and not switchSemaforo:
                        self.siguienteEstadoS=self.AMARILLO
                        
                    else: 
                        self.siguienteEstadoS=self.ROJO

    #Se generan los cambios asignados en la función step
    def advance(self):
        self.estadoS=self.siguienteEstadoS


#Modelo-------------------------------------------------------------------------------------------------------------------------------------------------------------
class Habitacion(Model):
    #Constructor del modelo, el cual inicializa sus valores
    def __init__(self, m=10, n=10, num_agentes=4):
        self.num_agentes = num_agentes
        self.grid = MultiGrid(m, n, True)
        self.schedule = SimultaneousActivation(self)
               
        #Crear la calle/interseccion del modelo
        auxM2=(m/2)-1
        auxM1=(m/2)
        auxN2=(n/2)-1
        auxN1=(n/2)
        for celdas1 in range(m):
            for celdas2 in range(n):
                celda_posicion=(celdas1,celdas2)
                if celdas1 == auxM2 and celdas2 == auxN2:
                    piso= Piso(celda_posicion, self)
                    piso.estado=piso.CALLE
                    piso.SENTIDO_CALLE=piso.SENTIDO_TODOS
                    self.grid.place_agent(piso, celda_posicion)
                    self.schedule.add(piso)
                    continue

                if celdas1 == auxM1 and celdas2 == auxN1:
                    piso= Piso(celda_posicion, self)
                    piso.estado=piso.CALLE
                    piso.SENTIDO_CALLE=piso.SENTIDO_TODOS
                    self.grid.place_agent(piso, celda_posicion)
                    self.schedule.add(piso)
                    continue

                if celdas1 == auxM2 and celdas2 == auxN1:
                    piso= Piso(celda_posicion, self)
                    piso.estado=piso.CALLE
                    piso.SENTIDO_CALLE=piso.SENTIDO_TODOS
                    self.grid.place_agent(piso, celda_posicion)
                    self.schedule.add(piso)
                    continue

                if celdas1 == auxM1 and celdas2 == auxN2:
                    piso= Piso(celda_posicion, self)
                    piso.estado=piso.CALLE
                    piso.SENTIDO_CALLE=piso.SENTIDO_TODOS
                    self.grid.place_agent(piso, celda_posicion)
                    self.schedule.add(piso)
                    continue

                if celdas1 == auxM2:
                    piso= Piso(celda_posicion, self)
                    piso.estado=piso.CALLE
                    piso.SENTIDO_CALLE=piso.SENTIDO_Oeste
                    self.grid.place_agent(piso, celda_posicion)
                    self.schedule.add(piso)
                    continue

                if celdas1 == auxM1:
                    piso= Piso(celda_posicion, self)
                    piso.estado=piso.CALLE
                    piso.SENTIDO_CALLE=piso.SENTIDO_Este
                    self.grid.place_agent(piso, celda_posicion)
                    self.schedule.add(piso)
                    continue

                if celdas2 == auxN2:
                    piso= Piso(celda_posicion, self)
                    piso.estado=piso.CALLE
                    piso.SENTIDO_CALLE=piso.SENTIDO_Norte
                    self.grid.place_agent(piso, celda_posicion)
                    self.schedule.add(piso)
                    continue

                if celdas2 == auxN1:
                    piso= Piso(celda_posicion, self)
                    piso.estado=piso.CALLE
                    piso.SENTIDO_CALLE=piso.SENTIDO_Sur
                    self.grid.place_agent(piso, celda_posicion)
                    self.schedule.add(piso)
                    continue
        
        # Posicionar celdas donde no puede ir el coche (casas, edificios, etc)
        lista_celdas_vacias = list(self.grid.empties)
        for celdas in lista_celdas_vacias:
            if celdas==(int(auxM2-1),int(auxN2-1)) or celdas==(int(auxM2-1),int(auxN1+1)) or celdas==(int(auxM1+1),int(auxN2-1)) or celdas==(int(auxM1+1),int(auxN1+1)):
                continue
            else:
                piso = Piso(celdas, self)
                self.grid.place_agent(piso, celdas)
                self.schedule.add(piso)
        
        # Posicionar coches
        i=0
        for cocheP1 in range(m):
            for cocheP2 in range(n):
                if i<self.num_agentes:
                    if cocheP1==auxM1 and cocheP2==0:
                        i=i+1
                        coche = Coche(i, self)
                        self.grid.place_agent(coche, (cocheP1,cocheP2))
                        self.schedule.add(coche)
                        continue
                    if cocheP1==0 and cocheP2==auxN2:
                        i=i+1
                        coche = Coche(i, self)
                        self.grid.place_agent(coche, (cocheP1,cocheP2))
                        self.schedule.add(coche)
                        continue
                    if cocheP1==m-1 and cocheP2==auxN1:
                        i=i+1
                        coche = Coche(i, self)
                        self.grid.place_agent(coche, (cocheP1,cocheP2))
                        self.schedule.add(coche)
                        continue
                    if cocheP1==auxM2 and cocheP2==n-1:
                        i=i+1
                        coche = Coche(i, self)
                        self.grid.place_agent(coche, (cocheP1,cocheP2))
                        self.schedule.add(coche)
                        continue
                else:
                    break

        #Pocisionar semáforos
        i=0
        posiciones_semaforo=[(int(auxM2-1),int(auxN2-1)),
                     (int(auxM2-1),int(auxN1+1)),
                     (int(auxM1+1),int(auxN2-1)),
                     (int(auxM1+1),int(auxN1+1))]

        for semas in posiciones_semaforo:
            i=i+1
            semaforo = Semaforo(semas, self)
            if i==1:
                semaforo.sentidoSEMAFORO="N"
            if i==2:
                semaforo.sentidoSEMAFORO="O"
            if i==3:
                semaforo.sentidoSEMAFORO="E"
            if i==4:
                semaforo.sentidoSEMAFORO="S"
            self.grid.place_agent(semaforo, semas)
            self.schedule.add(semaforo)


        self.colectordatos = DataCollector(
            model_reporters={'Habitacion': obtener_habitacion},
            agent_reporters={'Movimientos': lambda a: getattr(a, 'movimientos', None)}
        )
    
    def step(self):
        self.colectordatos.collect(self)
        self.schedule.step()    
  

## Correr el modelo

In [4]:
# Tiempo máximo de ejecución (segundos)
TIEMPO_MAXIMO_EJECUCION = 0.5

start_time = time.time()
tiempo_inicio = str(datetime.timedelta(seconds=TIEMPO_MAXIMO_EJECUCION))
modelo = Habitacion(20,20,4)

while((time.time() - start_time) < TIEMPO_MAXIMO_EJECUCION):
    modelo.step()

# Obtenemos el tiempo que le tomó correr al modelo.
tiempo_ejecucion = str(datetime.timedelta(seconds=(time.time() - start_time)))

## Visualización

In [5]:
todas_habitaciones = modelo.colectordatos.get_model_vars_dataframe()

In [6]:
%%capture

colors = [(1,1,1), (128/255, 128/255, 128/255), (1, 0, 0), (1,1,0), (0,1,0),(0,0,0)]  
cmap_name = 'my_list'
cmap = LinearSegmentedColormap.from_list(cmap_name, colors, 6)

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(todas_habitaciones.iloc[0][0], cmap=cmap)

def animate(i):
    patch.set_data(todas_habitaciones.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=len(todas_habitaciones))

In [7]:
anim

## Informe

In [8]:
movimientos = modelo.colectordatos.get_agent_vars_dataframe()

print('Tiempo necesario hasta que todas las celdas estén limpias:', tiempo_ejecucion, '/', tiempo_inicio)
print('Número de movimientos realizados por todos los agentes: ', movimientosT)

Tiempo necesario hasta que todas las celdas estén limpias: 0:00:00.500649 / 0:00:00.500000
Número de movimientos realizados por todos los agentes:  1244
